# Equality Constrained Entropy Maximization


This is an exercise from [Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/)

$$ \text{minimize } \qquad f(x) = \displaystyle\sum_{i=1}^n x_i\log(x_i) $$

$$ \text{subject to } \qquad Ax=b $$

## Feasible Start

For now, we assume that we have a feasible point $x$, so we assume that we have a point $x$ such that $Ax=b$.

Let's replace the objective with its second order Taylor Approximation at $x$.

$$ \hat{f}(v) = f(x) + \nabla f(x)^T (v-x) + \frac{1}{2} (v-x)^T \nabla^2 f(x) (v-x) $$

$$ \text{minimize } \qquad \hat{f}(x+v) = \frac{1}{2} v^T \nabla^2 f(x) v + \nabla f(x)^Tv + f(x)$$

$$ \text{subject to} \qquad A(x+v)=b$$

with respect to the variable $v$. We can solve this problem and we think of the solution as the value which needs to be added to $x$ to minimize the quadratic approximation. This value is the Newton step.

We can solve the quadratic approximation minimization problem (and therefore find the Newton step) by forming the dual and solving an unconstrained minimization problem.

$$ \text{minimize } \qquad \frac{1}{2} v^T \nabla^2 f(x) v + \nabla f(x)^T v + f(x) + \lambda^T(A(x+v)-b) $$

A solution to this problem is equivalent to finding a solution $v^{\star}, \lambda^{\star}$ to the KKT equations

$$A(x+v^{\star})=b \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} + \nabla f(x) + A^T \lambda^{\star} = 0$$

Note that we compute the gradients with respect to the variable $v$. 

We re-write the equations

$$Av^{\star}=b-Ax = 0 \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} +  A^T \lambda^{\star} = -\nabla f(x)$$


We can compute the Newton step by solving the KKT system

$$ \begin{bmatrix} \nabla^2f(x) & A^T \\ A & 0 \end{bmatrix} \begin{bmatrix} v^{\star} \\ \lambda^{\star} \end{bmatrix}  = \begin{bmatrix} -\nabla f(x) \\ 0 \end{bmatrix}$$

where $\Delta x_{nt} = v^{\star}$ is the Newton step (the step we use to update our current feasible point) and $w = \lambda^{\star}$ is the updated dual variable (already updated, no step necessary).

In [1]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
import scipy

In [2]:
# generate problem data
n = 100
p = 30
A = np.random.normal(0,1,(p,n))
x_hat = np.random.uniform(0,10,100) # use this as feasible intial point
b = A@x_hat 

In [3]:
# solve KKT system via block elimination
def KKT_solve(H,A,g,h):
    H_inv = np.linalg.inv(H)
    X = H_inv@A.T
    y = H_inv@g
    S_inv = np.linalg.inv(-A@X)
    w = S_inv@(A@y-h)
    return H_inv@(-A.T@w-g), w
def l2_norm(x):
    return np.sqrt(np.sum(x**2))
def entropy(x):
    return np.sum(x*np.log(x))
def grad(x):
    return np.log(x)+1
def hessian(x):
    return np.diag(1/x)
def backtrack(x, objective, alpha, beta, grad, descent_direction):
    t = 1
    while np.min(x+t*descent_direction)<=0: # check that updated point will be in the domain
        t = t*beta
    while objective(x+t*descent_direction) > objective(x) + alpha*t*(grad@descent_direction):
        t = t*beta
    return t

In [4]:
def Newtons_method(obj, A, b, x_init, grad_func, hessian_func, tol=1e-8, max_iter=1000, alpha=0.1, beta=0.5 ):
    x = x_init
    i = 0
    if (A@x-b == 0).all():
        print('Feasible start')
        while i <= max_iter:
            g = grad_func(x)
            H = hessian_func(x)
            delta_x, w = KKT_solve(H,A,g,np.zeros(A.shape[0]))
            dec = delta_x@(H@delta_x)
            if dec < tol:
                break
            else:
                t = backtrack(x, obj, alpha, beta, g, delta_x)
                x = x+t*delta_x
            i += 1
    else:
        print('Infeasible Start')
        w = np.zeros(A.shape[0])
        while i <= max_iter:
            g = grad_func(x)
            H = hessian_func(x)
            delta_x, updated_w = KKT_solve(H,A,g,A@x-b)
            delta_w = updated_w - w
            t = r_backtrack(x,w,delta_x,delta_w,A,b, alpha, beta)
            x = x+t*delta_x
            w = w+t*delta_w
            if l2_norm(A@x-b) < tol and l2_norm(r(x,w,A,b))< tol:
                break
            i += 1
            
    return x

In [5]:
# feasible start example
solution = Newtons_method(entropy, A,b, x_hat, grad, hessian)
entropy(solution)

Feasible start


322.0858921942636

## Infeasible Start

Previously, we have seen how to find the Newton step by solving the KKT equations

$$Av^{\star}=b-Ax = 0 \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} +  A^T \lambda^{\star} = -\nabla f(x)$$

Note that if the point $x$ is infeasible, $Ax-b \neq 0$. So the KKT system becomes

$$ \begin{bmatrix} \nabla^2f(x) & A^T \\ A & 0 \end{bmatrix} \begin{bmatrix} v^{\star} \\ \lambda^{\star} \end{bmatrix}  = -\begin{bmatrix} \nabla f(x) \\ Ax-b \end{bmatrix}$$

We want to update the dual variable as well to approximately satisfy the optimality conditions. From the KKT conditions for the original problem, we want 

$$ r_{\text{dual}} (x, \lambda) = \nabla f(x) + A^T \lambda \qquad \qquad r_{\text{primal}}(x, \lambda) = Ax-b$$

the dual and primal residuals to be 0. So, we want to update the primal and dual variables to drive the entries of 

$$ r(x ,\lambda) = (r_{\text{dual}} (x, \lambda),r_{\text{primal}}(x, \lambda)) $$

to 0. 

In [6]:
def r(x,w,A,b):
    return np.append(grad(x)+A.T@w, A@x-b)
def r_backtrack(x,w,delta_x,delta_w,A,b, alpha, beta):
    t = 1
    l2 = l2_norm(r(x,w,A,b))
    while np.min(x+t*delta_x) <= 0: # the gradient involves log, need to have updated point in domain
        t = t*beta
    while l2_norm(r(x+t*delta_x, w+t*delta_w,A,b)) > (1-alpha*t)*l2:
        t = beta*t
    return t

In [7]:
entropy(Newtons_method(entropy, A, b, np.ones(A.shape[1]),grad, hessian))

Infeasible Start


322.0858921942287

In [8]:
x = cp.Variable(n)
objective = cp.Minimize(cp.sum(-1*cp.entr(x)))
constraints = [A@x==b]
prob = cp.Problem(objective, constraints)
prob.solve()

322.0858909544767

## Solve the dual of the original problem

Original Problem

$$ \text{minimize } \qquad f(x) = \displaystyle\sum_{i=1}^n x_i\log(x_i) $$

$$ \text{subject to } \qquad Ax=b $$

Dual Problem

$$ \text{maximize } \qquad g(\lambda) = \text{inf} \left( f(x) + \lambda^T(Ax-b)\right)$$

We can express the dual function $g$ using the conjugate $f^{*}$ of the objective function $f$:

$$g(\lambda) = -b^T\lambda - f^{*} (-A^T \lambda)$$

The conjugate of negative entropy is 

$$ f^{*} (y) = \displaystyle\sum_{i=1}^n \text{exp}(y_i-1) $$

Our goal is to solve the problem

$$ \text{maximize} \qquad -b^T\lambda - \displaystyle\sum_{i=1}^n \text{exp}(-a_i^T\lambda - 1)$$

where $a_i$ is the $i$th column of the matrix $A$. We solve the equivalent problem

$$ \text{minimize} \qquad b^T\lambda + \displaystyle\sum_{i=1}^n \text{exp}(-a_i^T\lambda - 1)$$

with respect to the variable $\lambda$.

In [9]:
def dual_backtrack(x, objective,A, b, alpha, beta, gradient, descent_direction):
    t = 1
    while objective(x+t*descent_direction, A, b) > objective(x,A,b)+alpha*t*gradient@descent_direction:
        t = t*beta
    return t
def dual_grad(x,A,b):
    return b-A@(np.exp(-A.T@x-1))
def dual_hessian(x, A, b):
    return A@np.diag(np.exp(-A.T@x-1))@A.T
def dual_objective(x,A,b):
    return b@x + np.sum(np.exp(-A.T@x-1))

In [10]:
# Newton's Method
z = np.zeros(A.shape[0])
tol = 1e-8 
max_iter = 100
alpha = 0.1 # line search
beta = 0.5 # line search
i = 0
while i <= max_iter:
    g = dual_grad(z, A, b)
    H = dual_hessian(z, A, b)
    newton_step = -np.linalg.inv(H)@g
    dec = -g@newton_step
    if dec < tol:
        break
    else:
        t = dual_backtrack(z, dual_objective,A,b,alpha, beta, g, newton_step)
        z = z+t*newton_step
    i +=1

In [11]:
-dual_objective(z,A,b)

322.08589219422845

Suppose we find a solution $\lambda^*$ to the dual problem. Then, if we can find a point $x^*$ which minimizes the Lagrangian $L(x,\lambda^*)$, then $x^*$ is a primal optimal point. 

$$L(x, \lambda^*) = \displaystyle\sum_{i=1}^n x_i\log(x_i) + A^T \lambda^* $$

We can compute the gradient, set its entries equal to 0, and solve to find an optimal primal point.

$$ [\nabla L(x, \lambda^*)]_i = \log(x_i) + 1 + a_i^T\lambda^* = 0 $$

$$ x_i = \text{exp}(-a_i^T-1) $$

In [12]:
np.exp(-A.T@z-1)

array([ 4.51172304,  0.11104154,  0.74921668,  4.76127343,  1.724286  ,
        0.01688706,  5.80700802,  1.99563021,  0.17857001,  0.50256622,
        0.0316494 ,  0.20322001,  1.93129785, 10.65697209,  0.44414863,
        0.57036473,  0.09401531,  0.04830183,  0.08493813,  4.81811148,
        0.42948697,  0.01875593,  1.82439404,  0.01213289,  3.73511001,
        7.21658943,  0.59360135,  0.70789631,  3.94803617,  2.05566878,
        4.23101779,  0.30382158,  0.15811449,  1.96515233,  0.1052523 ,
        0.57520529,  1.74613686,  0.10572014,  0.02771708,  0.01422394,
        2.81932953,  1.10303362,  0.3531199 ,  0.10156372,  0.39657253,
        0.24769834,  0.64490116,  0.01242223,  3.64925267,  5.20474737,
        0.84266314, 10.09545141,  2.27315201,  2.88416505,  1.40057728,
        4.00911148,  1.80183725,  3.5669103 ,  4.10699365,  0.39067512,
        8.68317653,  4.98383894,  0.35890291,  0.51947781,  0.35877177,
        0.26927109,  0.4111495 ,  0.44138242,  0.63240276, 10.98

In [13]:
z = cp.Variable(30)
objective = cp.Maximize(-b@z-cp.sum(cp.exp(-A.T@z-1)))
prob = cp.Problem(objective,[])
prob.solve()

322.0858913948191

In [14]:
z.value

array([-8.97371719e-02,  1.31940762e+00,  4.38237874e-01, -3.13514754e-02,
       -3.17440568e-01,  2.49960286e-01,  3.13720137e-01,  1.94684184e-01,
        1.41697746e-01, -1.96065750e-01, -7.97446997e-01, -1.80909804e-01,
       -6.26185707e-02,  4.52818455e-04,  1.02376145e-01,  5.69996605e-01,
       -1.57409659e-01,  3.32849376e-01,  1.25864141e-01, -2.40973023e-01,
       -5.06402843e-03, -1.97574257e-01,  3.49730252e-01,  9.30316445e-01,
        4.73586211e-01,  3.76327151e-01,  2.15718684e-01,  4.83034712e-02,
       -3.07036702e-02,  3.12644529e-01])